In [1]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
import re

nltk.download('stopwords')
stopwords = set(stopwords.words('english'))
from dateutil import parser as dateParser
from datetime import date
from datetime import datetime

import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np

# from timezonefinder import TimezoneFinder
# from dateutil import tz

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/brijeshpatel/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


ModuleNotFoundError: No module named 'geocoder'

In [ ]:
df = pd.read_json("~/Desktop/ufoReports.json", orient="records", dtype=object, convert_dates=False)
df.info()

In [ ]:
# There were some redundant columns so dropping it
df = df.drop(columns=df.columns[7:13])
df.info()

In [ ]:
#checking for duplicates,
duplicate_index = df[df.duplicated(["Occurred", "Location", "Characteristics"])].index
print(f'number of rows={len(df)}, duplicated rows={len(duplicate_index)}')
df = df.drop(duplicate_index)
print(f'number of rows after deletion: {len(df)}')

In [ ]:
# Covert date time to proper format
# used "(O|R|P).*":\s"\s*\d{4} to check date formate in data
def convertDateTime(text):
    try:
        # date time AM or PM if it is mentioned
        dateTimePattern = re.compile('.*\d{1,2}/\d{1,2}/\d{4} \d{1,2}:\d{1,2}(:\d{1,2})? (AM|PM)?')
        # timeZone = tz.gettz(TimezoneFinder().timezone_at(latlang))
        stringTime = dateTimePattern.search(text).group()
        return dateParser.parse(stringTime) if stringTime else None
    except Exception as e:
        return None

df['TimeOfEvent'] = df.Occurred.apply(convertDateTime)
df['ReportedTime'] = df.Reported.apply(convertDateTime)

In [ ]:
# parse Duration to seconds
def calculateSeconds(string):
    try:
        dt = date.today()
        dt = datetime.combine(dt, datetime.min.time())
        dn = dateParser.parse(string)
        timeString = str(dn - dt)
        h, m, s = map(int, timeString.split(':'))
        return h * 3600 + m * 60 + s
    except Exception as e:
        return None

df['DurationInSecond'] = df.Duration.apply(calculateSeconds)

In [ ]:
# fill in missing shape value
def fillShape(x):
    x = x.strip()
    if (x):
        x = x.strip()
        if(x.lower()=="other"):
            return "Unknown"
        else:
            return x
    else:
        return "Unknown"

df['Shape'] = df['Shape'].apply(fillShape)

In [ ]:
# Remove rows for which transformation did not apply
rbt = df.shape
df = df.dropna(subset=["TimeOfEvent","ReportedTime","Shape"], axis=0, how="any")
df.drop(columns=["Posted","Occurred","Reported","Duration"], axis=1 )
rn= df.shape
print(df.info())
print(f"Before:{rbt}, after:{rn}")

In [ ]:
# Clean Duration
sns.boxplot(x=df['DurationInSecond'])
plt.show()

In [ ]:
# Sctter plt for duration
fig, ax = plt.subplots(figsize=(16,8))
ax.scatter(df['TimeOfEvent'], df['DurationInSecond'])
ax.set_xlabel('Time when UFO appeared')
ax.set_ylabel('Duation')
plt.show()

In [ ]:
# calculate z score to find outliers in Duration
z = np.abs(stats.zscore(df["DurationInSecond"], nan_policy='omit'))
# as common practice, remove z where abs(z) > 3
delete_indexes = z[z>3].index
df = df.drop(delete_indexes, axis=0)

#replace null duration with mean
meanDuration = df['DurationInSecond'].mean()
df['DurationInSecond'] = df['DurationInSecond'].fillna(meanDuration)
del(df['Duration'])
df.info()

In [ ]:
fig, ax = plt.subplots(figsize=(16,8))
ax.scatter(df['TimeOfEvent'], df['DurationInSecond'])
ax.set_xlabel('Time when UFO appeared')
ax.set_ylabel('Duation')
plt.show()

In [ ]:
# create a column to summarize characteristics or user report
pattern = re.compile(r'\b(' + r'|'.join(stopwords) + r')(\b\s*)|("\\n")', re.IGNORECASE)
df['Summary'] = df.Characteristics.apply(lambda x: pattern.sub('', x) if isinstance(x, str) else "")

In [ ]:
# # using latitude longitude so it is easy to put on a map and find timeZone
# def getlatlang(loc):
#     try:
#         location = geocoder.arcgis(loc)
#         return location.latlng
#     except Exception as e:
#         print(e)
#         return None
# df["latlang"] = df.Location.apply(lambda x: getlatlang(x) if x else None)